In [20]:
import xgboost as xgb
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

In [2]:
df_pop = pd.read_csv('../../../data/popular_data.csv')
df_pop

,Unnamed: 0,infobox_key,page_title,property_name,template,previous_value,current_value,timestamp,days_diff
0,93,216369130-0,NCSoft,name,infobox company,NCSOFT Corporation<br/>엔씨소프트,NCsoft Corporation<br/>엔씨소프트,2017-01-15 16:52:10+00:00,606 days 15:01:36
1,94,216369130-0,NCSoft,name,infobox company,NCsoft Corporation<br/>엔씨소프트,NCSOFT Corporation<br/>엔씨소프트,2017-05-30 20:04:15+00:00,135 days 03:12:05
2,95,216369130-0,NCSoft,name,infobox company,NCSOFT Corporation<br/>엔씨소프트,NCSoft Corporation<br/>엔씨소프트,2018-09-30 11:38:58+00:00,487 days 15:34:43
3,663,27792315-0,Wildflowers (Tom Petty album),genre,infobox album,"[[Heartland rock]],[[ Hard rock]]",[[Heartland rock]],2015-07-10 09:16:12+00:00,661 days 05:46:00
4,664,27792315-0,Wildflowers (Tom Petty album),genre,infobox album,[[Heartland rock]],{{flatlist|\n* [[Heartland rock]]\n* [[rock an...,2015-09-03 15:28:03+00:00,55 days 06:11:51
...,...,...,...,...,...,...,...,...,...
6524481,64239164,134041145-0,François Truffaut,spouse,infobox person,{{marriage|Madeleine Morgenstern|1957|1965}} (...,{{marriage|Madeleine Morgenstern|1957|1965|end...,2016-02-06 21:31:24+00:00,963 days 15:58:29
6524482,64239165,134041145-0,François Truffaut,spouse,infobox person,{{marriage|Madeleine Morgenstern|1957|1965|end...,"Madeleine Morgenstern<br>(1957－1965, 2 daughters)",2016-05-20 12:28:44+00:00,103 days 14:57:20
6524483,64239166,134041145-0,François Truffaut,spouse,infobox person,"Madeleine Morgenstern<br>(1957－1965, 2 daughters)","Madeleine Morgenstern<br>(1957–1965, 2 daughters)",2017-09-07 20:57:16+00:00,475 days 08:28:32
6524484,64239167,134041145-0,François Truffaut,spouse,infobox person,"Madeleine Morgenstern<br>(1957–1965, 2 daughters)",{{marriage|Madeleine Morgenstern<br>|1957|1965...,2018-06-09 10:06:14+00:00,274 days 13:08:58


In [3]:
df_pop = df_pop[['infobox_key', 'property_name', 'template', 'timestamp','days_diff']]
df_pop.head(3)

,infobox_key,property_name,template,timestamp,days_diff
0,216369130-0,name,infobox company,2017-01-15 16:52:10+00:00,606 days 15:01:36
1,216369130-0,name,infobox company,2017-05-30 20:04:15+00:00,135 days 03:12:05
2,216369130-0,name,infobox company,2018-09-30 11:38:58+00:00,487 days 15:34:43


In [21]:

datetime.datetime.strptime(df_pop["timestamp"][0][:19],
                                  "%Y-%m-%d %H:%M:%S").timetuple().tm_yday


15

In [22]:
df_pop = df_pop[df_pop["template"] == "football player infobox"]
df_pop = df_pop[df_pop["property_name"] == "caps(goals)"]
df_pop

,infobox_key,property_name,template,timestamp,days_diff
1882,66193870-0,caps(goals),football player infobox,2007-08-09 00:44:41+00:00,60 days 13:21:29
1883,66193870-0,caps(goals),football player infobox,2008-03-17 04:46:11+00:00,221 days 04:01:30
1884,66193870-0,caps(goals),football player infobox,2008-06-28 17:40:31+00:00,103 days 12:54:20
1885,66193870-0,caps(goals),football player infobox,2008-10-14 14:45:17+00:00,107 days 21:04:46
1886,66193870-0,caps(goals),football player infobox,2008-12-29 15:45:11+00:00,76 days 00:59:54
...,...,...,...,...,...
6522923,82868005-0,caps(goals),football player infobox,2010-03-09 13:21:56+00:00,576 days 10:40:06
6522924,82868005-0,caps(goals),football player infobox,2010-07-09 05:54:47+00:00,121 days 16:32:51
6522925,82868005-0,caps(goals),football player infobox,2010-07-27 14:25:13+00:00,18 days 08:30:26
6522943,82870020-0,caps(goals),football player infobox,2008-07-25 05:48:03+00:00,414 days 12:52:13


In [23]:
# get boxes with >=10 changes
df_pop_many = df_pop.groupby('infobox_key')['property_name'].count().reset_index().rename(columns={'property_name':'count'})
pop_keys = df_pop_many[df_pop_many['count']>=10]['infobox_key'].tolist()

In [24]:
df_pop = df_pop[df_pop['infobox_key'].isin(pop_keys)]
df_pop.groupby("infobox_key").count()

,property_name,template,timestamp,days_diff
infobox_key,,,,
100003909-0,85,85,85,85
100006428-0,28,28,28,28
100026229-0,16,16,16,16
100031554-0,21,21,21,21
100074928-0,33,33,33,33
...,...,...,...,...
99964202-0,17,17,17,17
99977353-0,52,52,52,52
99977440-0,68,68,68,68


In [35]:
# convert timestamp to day of year
df_pop.loc[:,'day_of_year'] = df_pop.loc[:,'timestamp'].apply(lambda x:datetime.datetime.strptime(x[:19],
                                  "%Y-%m-%d %H:%M:%S").timetuple().tm_yday)

# calc time in days until next change (0 if last change)
df_pop.loc[:,'time_from_prev_change'] = df_pop.loc[:,'days_diff'].apply(lambda x: round(pd.Timedelta(x).total_seconds()/(3600*24)))
df_pop.loc[:,'time_til_next_change'] = df_pop.groupby(['infobox_key', 'property_name'])['time_from_prev_change'].shift(-1)
df_pop.loc[:,'time_til_next_change'] = pd.to_numeric(df_pop.loc[:,'time_til_next_change'].fillna(0),downcast="integer")
df_pop

/home/vpr/miniconda3/envs/mp/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/vpr/miniconda3/envs/mp/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


,infobox_key,property_name,template,timestamp,days_diff,day_of_year,time_from_prev_change,time_til_next_change
1882,66193870-0,caps(goals),football player infobox,2007-08-09 00:44:41+00:00,60 days 13:21:29,221,61,221
1883,66193870-0,caps(goals),football player infobox,2008-03-17 04:46:11+00:00,221 days 04:01:30,77,221,104
1884,66193870-0,caps(goals),football player infobox,2008-06-28 17:40:31+00:00,103 days 12:54:20,180,104,108
1885,66193870-0,caps(goals),football player infobox,2008-10-14 14:45:17+00:00,107 days 21:04:46,288,108,76
1886,66193870-0,caps(goals),football player infobox,2008-12-29 15:45:11+00:00,76 days 00:59:54,364,76,1
...,...,...,...,...,...,...,...,...
6522775,82855757-0,caps(goals),football player infobox,2009-08-30 20:25:53+00:00,12 days 09:31:14,242,12,9
6522776,82855757-0,caps(goals),football player infobox,2009-09-08 10:09:21+00:00,8 days 13:43:28,251,9,20
6522777,82855757-0,caps(goals),football player infobox,2009-09-28 10:28:55+00:00,20 days 00:19:34,271,20,29
6522778,82855757-0,caps(goals),football player infobox,2009-10-27 10:19:19+00:00,28 days 23:50:24,300,29,319


In [37]:
df_pop.drop(columns=['days_diff'], inplace=True)
df_pop.drop(columns=['timestamp'], inplace=True)
df_pop.head(3)

,infobox_key,property_name,template,day_of_year,time_from_prev_change,time_til_next_change
1882,66193870-0,caps(goals),football player infobox,221,61,221
1883,66193870-0,caps(goals),football player infobox,77,221,104
1884,66193870-0,caps(goals),football player infobox,180,104,108


In [38]:
# used to slice last 2 entries per group
def __groupby_slice( _grp, start=0, stop=None, step=1):
    '''
    Applies a slice to a GroupBy object
    '''
    return _grp.apply( lambda _df : _df.iloc[start:stop:step]).reset_index(drop=True)

In [39]:
train_set = __groupby_slice(df_pop.groupby('infobox_key'), 0, -2)
train_set.shape

(142115, 6)

In [40]:
test_set = __groupby_slice(df_pop.groupby('infobox_key'), -2)
test_set.shape

(11886, 6)

In [46]:
X_pop_train = pd.get_dummies(train_set["infobox_key"])
X_pop_train["time_from_prev_change"] = train_set['time_from_prev_change'].copy()
X_pop_train['day_of_year'] = train_set['day_of_year'].copy()

y_pop_train = train_set['time_til_next_change'].copy()


X_pop_test = pd.get_dummies(test_set["infobox_key"])
X_pop_test["time_from_prev_change"] = test_set['time_from_prev_change'].copy()
X_pop_test['day_of_year'] = test_set['day_of_year'].copy()

y_pop_test = test_set['time_til_next_change'].copy()


In [54]:
X_pop_train

,time_from_prev_change,day_of_year
0,14,25
1,32,57
2,6,63
3,15,78
4,117,195
...,...,...
142110,3,112
142111,3,115
142112,103,218
142113,47,265


In [55]:
X_pop_train= pd.DataFrame(train_set['time_from_prev_change'].copy())
X_pop_train['day_of_year'] = train_set['day_of_year'].copy()

y_pop_train = train_set['time_til_next_change'].copy()


X_pop_test = pd.DataFrame(test_set['time_from_prev_change'].copy())
X_pop_test['day_of_year'] = test_set['day_of_year'].copy()

y_pop_test = test_set['time_til_next_change'].copy()

In [56]:
clf_xgb_pop = xgb.XGBRegressor(objective='reg:squarederror', missing=0, seed=42, n_estimators=100,max_depth=10)
clf_xgb_pop.fit(X_pop_train, y_pop_train, verbose=True, early_stopping_rounds=10, eval_set=[(X_pop_test, y_pop_test)])

[0]	validation_0-rmse:74.09995
[1]	validation_0-rmse:72.53200
[2]	validation_0-rmse:72.10468
[3]	validation_0-rmse:72.23489
[4]	validation_0-rmse:72.46361
[5]	validation_0-rmse:72.79226
[6]	validation_0-rmse:73.06788
[7]	validation_0-rmse:73.25365
[8]	validation_0-rmse:73.51253
[9]	validation_0-rmse:73.67167
[10]	validation_0-rmse:73.76984
[11]	validation_0-rmse:73.86164
[12]	validation_0-rmse:73.93074


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=10, min_child_weight=1, missing=0,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [57]:
xgb.to_graphviz(clf_xgb_pop, num_trees=0, size='10,10')